In [ ]:
# Topological sort leyendo un grafo desde un archivo de texto.
# Entrada: nombre de archivo con líneas "u v" (enteros o tokens) indicando arista u -> v
# Salida: lista con orden topológico (si existe) o None si hay ciclo
# Complejidad (peor caso): O(V + E) en tiempo.

from collections import defaultdict, deque

def read_grafo(archivo):
  adj = defaultdict(list)
  indeg = defaultdict(int)
  nodes = set()
  with open(archivo, 'r', encoding='utf-8') as f:
    for line in f:
      line = line.strip()
      if not line or line.startswith('#'):
        continue
      parts = line.split()
      if len(parts) < 2:
        continue
      u, v = parts[0], parts[1]
      nodes.add(u); nodes.add(v)
      adj[u].append(v)
      indeg[v] += 1
      # Asegurar que u y v aparecen en indeg/adj
      if u not in indeg:
        indeg[u] = indeg.get(u, 0)
  # Asegurar que todos los nodos están en adj y indeg
  for n in nodes:
    adj.setdefault(n, [])
    indeg.setdefault(n, 0)
  return nodes, adj, indeg

def topo_sort(archivo):
  nodes, adj, indeg = read_grafo(archivo)
  q = deque()
  # push nodes with indegree 0
  for n in nodes:
    if indeg[n] == 0:
      q.append(n)
  L = []
  while q:
    v = q.popleft()
    L.append(v)
    # remover v y sus aristas
    for w in adj[v]:
      indeg[w] -= 1
      if indeg[w] == 0:
        q.append(w)
  if len(L) == len(nodes):
    return L
  else:
    # hay ciclo
    return None

if __name__ == "__main__":
  fn = "grafo.txt"
  res = topo_sort(fn)
  if res is None:
    print("El grafo tiene ciclos; no es posible un orden topológico.")
  else:
    print("Orden topológico:", res)

Orden topológico: ['1', '2', '3', '4', '5']


In [ ]:
# Fake-coin
# Entrada: coins = lista de números (p. ej. 1 para buena, 0 para falsa si lighter)
#         mode = 'lighter' o 'heavier' (si no se sabe, usa 'lighter' por defecto)
# Salida: índice (0-based) de la moneda falsa o None, y número de pesadas realizadas
# Complejidad (pesadas): O(log n) pesadas en el modelo de balanza.
# Nota: cada pesada hace sumas sobre subconjuntos (costo en tiempo depende de esos tamaños).

from typing import List, Tuple, Optional

def find_fake_simple(coins: List[float], mode: str = 'lighter') -> Tuple[Optional[int], int]:
    # Encuentra índice de la moneda falsa iterativamente.
    # Asume que 'mode' indica si la falsa es 'lighter' o 'heavier'.

    n = len(coins)
    if n == 0:
        return None, 0
    weigh_count = 0
    candidates = list(range(n))

    while len(candidates) > 2:
        m = len(candidates) // 2
        left = candidates[:m]
        right = candidates[m:m + m]   # aseguramos mismas longitudes
        remainder = candidates[m + m:]  # puede contener 0 o 1 elementos

        if not right:  # si no hay pareja, rompemos para manejar manualmente
            break

        weigh_count += 1
        s_left = sum(coins[i] for i in left)
        s_right = sum(coins[i] for i in right)

        if s_left == s_right:
            # si iguales, la falsa está en remainder (si existe), si no, tomamos left (arbitrario)
            candidates = remainder if remainder else left
        else:
            if mode == 'lighter':
              candidates = left if s_left < s_right else right
            else:  # 'heavier'
                candidates = left if s_left > s_right else right

    # Ahora quedan 1 o 2 candidatos (o 0 si lista vacía)
    if len(candidates) == 1:
        return candidates[0], weigh_count
    if len(candidates) == 2:
        weigh_count += 1
        i, j = candidates[0], candidates[1]
        if coins[i] == coins[j]:
            return None, weigh_count  # raro: ninguna parece falsa
        if mode == 'lighter':
            return (i if coins[i] < coins[j] else j), weigh_count
        else:
            return (i if coins[i] > coins[j] else j), weigh_count

    # si no se redujo (p. ej. lista pequeña), probar buscar linealmente
    for idx in candidates:
        # comparar con alguna otra moneda (si existe) para decidir
        others = [k for k in range(n) if k != idx]
        if not others:
            return idx, weigh_count
        weigh_count += 1
        s_idx = coins[idx]
        s_other = coins[others[0]]
        if s_idx != s_other:
            if mode == 'lighter':
                return (idx if s_idx < s_other else others[0]), weigh_count
            else:
                return (idx if s_idx > s_other else others[0]), weigh_count

    return None, weigh_count

# ---------- Ejemplos ----------
if __name__ == "__main__":
    # ejemplo: 15 monedas, la 7 es más ligera (0 en vez de 1)
    coins = [1]*15
    coins[7] = 0
    idx, w = find_fake_simple(coins, mode='lighter')
    print("Fake (lighter) en:", idx, "pesadas:", w)

    # ejemplo heavier
    coins2 = [1]*16
    coins2[11] = 2
    idx2, w2 = find_fake_simple(coins2, mode='heavier')
    print("Fake (heavier) en:", idx2, "pesadas:", w2)

    # ejemplo con valores booleanos (True=buena, False=falsa, tratamos False como 0)
    coins3 = [1 if x else 0 for x in [True, True, True, False, True]]
    idx3, w3 = find_fake_simple(coins3, mode='lighter')
    print("Fake (boolean) en:", idx3, "pesadas:", w3)


Fake (lighter) en: 7 pesadas: 3
Fake (heavier) en: 11 pesadas: 4
Fake (boolean) en: 3 pesadas: 2


In [ ]:
# Quick-select recursivo (decrease-and-conquer).
# Entrada: arr (lista), k (0-index) para elegir k-ésimo menor
# Salida: valor que es k-ésimo menor
# Complejidad peor caso: O(n^2).
import random

def partition(arr, left, right, pivot_index):
    pivot_value = arr[pivot_index]
    # mover pivot al final
    arr[pivot_index], arr[right] = arr[right], arr[pivot_index]
    store = left
    for i in range(left, right):
        if arr[i] < pivot_value:
            arr[store], arr[i] = arr[i], arr[store]
            store += 1
    # mover pivot a su posición final
    arr[store], arr[right] = arr[right], arr[store]
    return store

def quickselect(arr, k, left=0, right=None):
    if right is None:
        right = len(arr) - 1
    if left == right:
        return arr[left]
    # elegir pivote aleatorio para evitar caso degenerado en promedio
    pivot_index = random.randint(left, right)
    pivot_index = partition(arr, left, right, pivot_index)
    # posición del pivot
    if k == pivot_index:
        return arr[k]
    elif k < pivot_index:
        return quickselect(arr, k, left, pivot_index - 1)
    else:
        return quickselect(arr, k, pivot_index + 1, right)

if __name__ == "__main__":
    a = [7,2,1,6,8,5,3,4]
    k = 3
    val = quickselect(a.copy(), k)
    print(f"{k+1}-ésimo menor es {val}")

4-ésimo menor es 4


¿Los algoritmos para generar todas las permutaciones y subconjuntos de n elementos (algoritmos de fuerza bruta) son o no algoritmos decrease-and-conquer? ¿Por qué sí / por qué no? Si sí, ¿qué factor de decremento tienen?
Sí y no. Se puede implementar la generación de permutaciones/subconjuntos con recursión que reduce el tamaño en 1 en cada llamada (por ejemplo: para subconjuntos, decides incluir o no el primer elemento y recursas sobre n-1; para permutaciones extraes un elemento y permutas los n-1 restantes). En ese sentido son decrease-and-conquer con factor de decremento 1 (es decir, cada llamada recursiva trabaja con n-1).

Entonces, ¿estos algoritmos son también búsqueda exhaustiva o no?
Sí, son búsqueda exhaustiva porque exploran todas (o la gran mayoría) de las combinaciones posibles para resolver el problema. El objetivo allí no es reducir a un solo subproblema y terminar, sino enumerar todas las soluciones; por eso se consideran fuerza bruta / exhaustive search.